# 非同期ショッピングアシスタントの LangGraph 活用

## 概要

このチュートリアルでは、ユーザとの会話を維持しながら Amazon で商品を検索できる非同期ショッピングアシスタントを構築します。このユースケースは、Amazon Bedrock AgentCore の非同期機能と LangGraph を活用して、時間のかかるタスクをバックグラウンドで実行できる応答性の高いエージェントを作成する方法を示しています。

### ユースケースの詳細

| 情報                | 詳細                                                                         |
|---------------------|------------------------------------------------------------------------------|
| ユースケースのタイプ | ショッピングアシスタント                                                    |
| エージェントのタイプ | 非同期                                                                       |
| エージェントフレームワーク | LangGraph                                                             |
| LLM モデル          | Anthropic Claude 3 Haiku                                                     |
| コンポーネント       | AgentCore Runtime、Browser Tool、Async Tasks                                |
| 例の複雑さ           | 中級                                                                         |
| 使用した SDK         | Amazon BedrockAgentCore Python SDK                                           |

### ユースケースのアーキテクチャ

このショッピングアシスタントは、時間のかかるタスクを処理しながら会話の流れを維持できる応答性の高い AI エージェントを構築するための強力なパターンを示しています。

1. ユーザが興味のある商品について尋ねる
2. エージェントは AgentCore の非同期機能を使ってバックグラウンドタスクを開始する
3. バックグラウンドタスクが実行されている間、エージェントはユーザとの会話を続けることができる
4. 結果が出そろったら、エージェントがアクセスできるファイルに保存される
5. エージェントは適切なタイミングで結果を取得して提示できる

このアーキテクチャでは、以下のような主要コンポーネントが組み合わされています。
- **LangGraph**: エージェントのワークフローと意思決定プロセスを構築するため
- **AgentCore Runtime**: エージェントをホストおよびスケーリングするため
- **AgentCore Async Tasks**: バックグラウンド操作を管理するため
- **Browser Tool**: 商品情報を検索および抽出するため
- **ファイルシステム**: 検索結果を保存および取得するため

![](images/architecture_diagram.png)

### 主な機能

* **非同期タスク管理**: 会話をブロックしないバックグラウンドタスクを起動
* **Browser Tool 統合**: AgentCore の Browser Tool を使って商品情報を検索および抽出
* **ステートフル会話**: LangGraph を使って複数ターンにわたるコンテキストを維持
* **ファイルシステム統合**: ファイルシステムツールを使って検索結果を保存および取得
* **並列処理**: 複数の商品検索を同時に処理

## 前提条件

このチュートリアルを実行するには、以下が必要です:

* Python 3.10+
* 適切な権限を持つ AWS 認証情報
* Amazon Bedrock AgentCore SDK
* LangGraph
* Docker の実行 (ローカルテストとデプロイ用)

In [ ]:
# 必要なパッケージをインストールする

 pip install numpy
 pip install pandas
 pip install matplotlib
 pip install scikit-learn
!pip install bedrock-agentcore-sdk langgraph langchain langchain_aws typing_extensions

## 非同期エージェントの理解

従来の会話型エージェントは、ユーザーのリクエストを同期的に処理します。つまり、タスクが完了するまで会話がブロックされます。この方式は、簡単な操作には適していますが、Web ブラウジングや複雑な検索などの時間のかかるタスクでは、ユーザーエクスペリエンスが低下します。

非同期エージェントは、次の方法でこの問題を解決します。

1. **ノンブロッキング操作**: タスクはバックグラウンドで実行され、会話は継続される
2. **タスク管理**: エージェントは、バックグラウンドタスクの開始、監視、結果の取得ができる
3. **応答性の向上**: ユーザーは即座にフィードバックを受け取り、対話を続けることができる

Amazon Bedrock AgentCore は、タスク管理機能を通じて非同期操作をビルトインでサポートしており、この買い物アシスタントでもこの機能を活用します。

### 買い物アシスタントのユースケース

私たちの買い物アシスタントは以下のことを行います。
- ユーザーから商品検索クエリを受け取る
- Browser Tool を使って Amazon.com でバックグラウンドタスクを起動し検索を行う
- 検索中もユーザーとの会話を継続する
- 検索が完了したら結果をファイルに保存する
- ユーザーから要求があれば検索結果を取得して提示する

この手順は、商品検索に時間がかかるが、ユーザーは応答性のある対話を期待する EC アプリケーションにとって特に有用です。

## 非同期ショッピングアシスタントコードの理解

非同期ショッピングアシスタントの実装の主要コンポーネントを確認しましょう。以下のコードは、ユーザーとの会話を維持しながら Amazon で商品を検索できるショッピングアシスタントを実装しています。

**主要コンポーネント:**

1. **AgentCore 統合**: エージェントをホストする BedrockAgentCoreApp を初期化します
2. **ブラウザツール実装**: NovaAct を使用して、商品検索のためのブラウザを制御します
3. **非同期タスク管理**: 会話をブロックしない背景タスクを管理します
4. **LangGraph エージェント**: エージェントのワークフローと意思決定プロセスを定義します
5. **エントリポイント関数**: ストリーミングサポートを備えたエージェントの呼び出しを処理します

以下が非同期ショッピングアシスタントの完全な実装です:

In [ ]:
# エントリーポイントコードを構文の強調表示で表示する

日本語訳:

# エントリーポイントコードを構文強調表示で表示します。
!pygmentize async_example_langgraph_browser_tool.py

### エントリーポイントファイルの理解

`async_example_langgraph_browser_tool.py` ファイルには、非同期のショッピングアシスタントの完全な実装が含まれています。主要なコンポーネントを見ていきましょう。

1. **AgentCore 統合**:
   ```python
   from bedrock_agentcore.runtime import BedrockAgentCoreApp
   app = BedrockAgentCoreApp()
   ```
   これにより、エージェントをホストする AgentCore アプリケーションが初期化されます。

2. **ブラウザツールの実装**:
   ```python
   def _run_browser_task(request: str):
       with browser_session('us-west-2') as client:
           # ブラウザセッションのセットアップ
           # ...
           task_id = app.add_async_task("using_browser_tool")
           # ...
           with NovaAct(...) as nova_act:
               result = nova_act.act(prompt=request, max_steps=5)
               # ...
           success = app.complete_async_task(task_id)
   ```
   この関数は、NovaAct と AgentCore の非同期タスク管理を使用して、バックグラウンドでブラウザタスクを実行します。

3. **LangGraph エージェント**:
   ```python
   graph_builder = StateGraph(State)
   graph_builder.add_node("chatbot", chatbot)
   graph_builder.add_node("tools", tool_node)
   # ...
   graph = graph_builder.compile(checkpointer=checkpointer, store=in_memory_store)
   ```
   これにより、chatbot ノードとツールノードを持つ LangGraph エージェントが作成されます。

4. **非同期エントリーポイント**:
   ```python
   @app.entrypoint
   async def agent_invocation(payload, context):
       # ...
       async for chunk in graph.astream(tmp_msg, stream_mode="updates", config=config):
           # チャンクの処理と出力
   ```
   この関数は、`@app.entrypoint` でデコレートされた、エージェントのエントリーポイントです。async/await パターンを使用して、レスポンスをストリーミングします。

このファイルは、バックグラウンドタスクを実行しながら、ユーザーとの会話を維持できる非同期エージェントの構築方法を示しています。AgentCore のタスク管理機能と LangGraph の状態管理を活用して、レスポンシブなショッピングアシスタントを作成しています。

### requirements.txt ファイルの作成

次に、エージェントが必要とするすべての依存関係を列挙した requirements.txt ファイルを作成する必要があります。

日本語訳:
エージェントが必要とする依存関係をリストアップした requirements.txt ファイルを作成する必要があります。

 rasa==3.3.2
 rasa-sdk==3.3.0
 scipy==1.9.3
 scikit-learn==1.1.3
 aiohttp==3.8.3
 aiofiles==0.8.0
 fastapi==0.88.0
 uvicorn==0.20.0
 python-multipart==0.0.5

このファイルには、プロジェクトで使用するライブラリとそのバージョンが記載されています。 requirements.txt ファイルを作成することで、プロジェクトの依存関係を明確にし、他の人がプロジェクトをセットアップする際に役立ちます。

In [ ]:
%%writefile requirements.txt
bedrock-agentcore-sdk
langgraph
langchain
langchain_aws
typing_extensions
langchain_community

### ランタイムロールの作成

私たちのエージェントをデプロイする前に、AgentCore Runtimeで使用されるIAMロールを作成する必要があります。このロールには、Amazon Bedrock や他のAWS サービスにアクセスするための必要な許可が付与されます。

日本語訳:
### ランタイムロールの作成

エージェントをデプロイする前に、AgentCore Runtime で使用される IAM ロールを作成する必要があります。このロールには、Amazon Bedrock やその他の AWS サービスにアクセスするための必要な許可が付与されます。

In [ ]:
import sys
import os
import json
import boto3

# Get the current notebook's directory

現在のノートブックのディレクトリを取得する

import os
cwd = os.getcwd()
print( "Current working directory is: " + cwd )

# Print the current working directory
print( "Current working directory is: " + cwd )

# Change directory
os.chdir( '/path/to/new/directory' )

# Get the new current working directory
new_cwd = os.getcwd()
print( "New current working directory is: " + new_cwd )
current_dir = os.path.dirname(os.path.abspath('__file__' if '__file__' in globals() else '.'))

# Navigate up to the utils.py location

utils.py の場所まで上へ移動します。
utils_dir = os.path.join(current_dir, '..', '..')
utils_dir = os.path.abspath(utils_dir)

# Add to sys.path

import sys
sys.path.append('/path/to/app/')

# 半角英数字の前後に半角スペースを挿入
text = "Hello 123 World 456"
new_text = ""
for char in text:
    if char.isdigit():
        new_text += " " + char + " "
    else:
        new_text += char
print(new_text)  # => "Hello 123 World 456"

# コードやコマンド、変数名、関数名などの技術的な用語は翻訳せず、そのまま残す
import numpy as np
x = np.array([1, 2, 3])
print(x)  # => [1 2 3]

日本語訳:

# sys.pathに追加する

import sys
sys.path.append('/path/to/app/')

# 半角英数字の前後に半角スペースを挿入する
text = "Hello 123 World 456"
new_text = ""
for char in text:
    if char.isdigit():
        new_text += " " + char + " "
    else:
        new_text += char
print(new_text)  # => "Hello 123 World 456"

# コードやコマンド、変数名、関数名などの技術的な用語はそのまま残す
import numpy as np
x = np.array([1, 2, 3])
print(x)  # => [1 2 3]
sys.path.insert(0, utils_dir)

from utils import create_agentcore_role

agent_name="async_shopping_assistant"
agentcore_iam_role = create_agentcore_role(agent_name=agent_name)

### AgentCore Runtimeのデプロイを設定する

次に、AgentCoreスターターツールキットを使用して、私たちのエージェントのデプロイを設定します。これにより、アプリケーションコードに基づいて Dockerfile が生成され、デプロイ準備が整えられます。

日本語訳:
### AgentCore Runtime の デプロイを設定する

これから AgentCore スターターツールキットを使って、私たちのエージェントの デプロイを設定します。これにより、アプリケーションコードに基づいて Dockerfile が生成され、すべてが デプロイの準備が整えられます。

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
import time

boto_session = Session()
region = boto_session.region_name
print(f"Using AWS region: {region}")

agentcore_runtime = Runtime()

response = agentcore_runtime.configure(
    entrypoint="async_example_langgraph_browser_tool.py",
    execution_role=agentcore_iam_role['Role']['Arn'],
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region
)
response

### AgentCore Runtimeへのエージェントの起動

エージェントの設定が完了したので、次は AgentCore Runtime にエージェントを起動しましょう。これにより、Amazon ECR リポジトリが作成され、エージェントが AgentCore Runtime にデプロイされます。

<div style="text-align:left">
    <img src="images/launch.png" width="75%"/>
</div>

In [ ]:
launch_result = agentcore_runtime.launch()
launch_result

### AgentCore ランタイムのステータスを確認する

私たちのエージェントの展開状況を確認しましょう。ステータスが "READY" になるまで待ちます。

日本語訳:

### AgentCore ランタイムのステータスを確認する

エージェントの展開状況を確認しましょう。ステータスが "READY" になるまで待ちます。

In [ ]:
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

### AgentCore Runtimeの呼び出し

私たちのエージェントが展開され、準備ができたので、製品検索クエリでそれを呼び出してみましょう。これにより、私たちのショッピングアシスタントの非同期機能が実証されます。

 ` curl -X POST -H "Content-Type: application/json" -d '{"query": "I need a new laptop for video editing"}' http://localhost:8000/query `

上記の curl コマンドを使用して、ローカルホストの 8000 ポートでリッスンしている AgentCore Runtime にクエリを送信します。応答は JSON 形式で返されます。

 ` {
  "result": "Received query: I need a new laptop for video editing. Working on finding the best laptop for your needs..."
} `

この初期応答は、クエリが受信され、エージェントがタスクに取り組んでいることを示しています。エージェントは非同期で動作するため、完了までに時間がかかる可能性があります。

In [ ]:
from IPython.display import Markdown, display

# 製品検索クエリでエージェントを呼び出す

日本語訳:

製品検索クエリでエージェントを呼び出すには、以下のように入力します。

```python
response = agent.run(" search_quality_reflection ", product_search_query)
```

この `agent.run()` 関数は、2つの引数を取ります。

1. `" search_quality_reflection "` は、エージェントに実行させるタスクの名前です。この場合は、製品検索の質を反映させるタスクです。

2. `product_search_query` は、ユーザーが入力した製品検索クエリです。例えば `"ノート PC 15 インチ"` などです。

`response` 変数には、エージェントの応答が格納されます。この応答は、製品検索の質に関する洞察や提案が含まれています。

この方法で、製品検索クエリに基づいて、エージェントの知識と推論能力を活用することができます。
invoke_response = agentcore_runtime.invoke({"prompt": "I'm looking for a good laptop for programming. Can you help me find some options?"})

# Display the response

レスポンスを表示します
response_text = json.loads(invoke_response['response'][0].decode("utf-8"))
display(Markdown(response_text))

### 非同期ショッピングアシスタントの動作を見る

製品検索クエリでエージェントを呼び出すと、バックグラウンドタスクが製品を検索しながら、あなたとチャットを続けます。実際の動作を見るには:

1. AWS コンソールに移動し、Amazon Bedrock セクションに行きます
![](images/go_to_bedrock_agentcore_console.png)

2. ブラウザツールタブで、エージェントのブラウザセッションが表示されます
   ![](images/browser_use_tab.png)

3. AgentCore が提供する組み込みのブラウザサンドボックスが見られます
   ![](images/aws_built_in_browser_sandbox.png)

4. 「View Live Session」をクリックすると、ブラウザの動作が見られます
   ![](images/click_view_live_session.png)

5. エージェントがブラウザを操作して製品を検索する様子を見てください
   ![](images/watch_the_agent_interact_with_browser.png)

エージェントは、製品検索がバックグラウンドで実行されている間も、あなたとの会話を続けられることがわかります。検索が完了したら、エージェントに結果を見せるよう尋ねることができます。

In [ ]:
# フォローアップの質問をする

日本語訳:

人工知能 (AI) は、 `follow_up_question()` 関数を使って、フォローアップの質問を生成することができます。この関数は、ユーザーの入力と対話の履歴に基づいて、適切なフォローアップの質問を提案します。 `context` 変数には、対話の文脈が含まれています。 `user_input` 変数には、ユーザーの最新の入力が含まれています。

```python
follow_up_question(context, user_input)
```

この関数は、自然言語処理 (NLP) と機械学習のテクニックを組み合わせて、適切なフォローアップの質問を生成します。生成された質問は、ユーザーに対して表示され、対話を継続するためのプロンプトとして機能します。

フォローアップの質問を生成することで、AI は対話をより自然で関連性の高いものにすることができます。これにより、ユーザーとの対話がより建設的で生産的になり、ユーザーの要求をよりよく理解することができます。
invoke_response = agentcore_runtime.invoke({"prompt": "While you're searching, can you tell me what features I should look for in a programming laptop?"})

# Display the response

レスポンスを表示する
response_text = json.loads(invoke_response['response'][0].decode("utf-8"))
display(Markdown(response_text))

では、検索結果の準備ができているかを確認しましょう:

In [ ]:
# 背景タスクが完了するのを待つ

日本語訳:
背景タスクが完了するのを待つために 1 時間待ちます。
time.sleep(10)

# 検索結果を要求する

日本語訳:

検索結果を要求するには、以下の手順を実行します。

1. ブラウザで検索エンジン (例: Google) を開きます。
2. 検索ボックスに検索語句 (キーワード) を入力します。
3. Enter キーを押すか、検索ボタンをクリックして検索を実行します。

検索エンジンは、入力した検索語句に関連するウェブページを検索し、検索結果のリストを表示します。検索結果には、ウェブページのタイトル、URL、簡単な説明が含まれています。

検索結果を絞り込むには、検索語句を変更したり、検索オプションを使用したりできます。検索オプションには、検索範囲の指定 (ウェブ、画像、ニュースなど)、日付範囲の指定、言語の指定などがあります。

より適切な検索結果を得るには、検索語句を適切に選択することが重要です。キーワードを組み合わせたり、フレーズを使用したりすることで、検索をより具体的にできます。また、検索演算子 (site:、inurl:、intitle: など) を使用すると、検索をさらに絞り込むことができます。
invoke_response = agentcore_runtime.invoke({"prompt": "Can you show me the laptop search results now?"})

# Display the response

レスポンスを表示する
response_text = json.loads(invoke_response['response'][0].decode("utf-8"))
display(Markdown(response_text))

### boto3 を使用してエージェントを呼び出す

boto3 を使用すると、プログラムでエージェントを呼び出すことができます。これは、エージェントを他のアプリケーションに統合する際に役立ちます。

In [ ]:
agent_arn = launch_result.agent_arn
agentcore_client = boto3.client(
    'bedrock-agentcore',
    region_name=region
)

boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    qualifier="DEFAULT",
    payload=json.dumps({"prompt": "Can you search for wireless headphones?"})
)

if "text/event-stream" in boto3_response.get("contentType", ""):
    content = []
    for line in boto3_response["response"].iter_lines(chunk_size=1):
        if line:
            line = line.decode("utf-8")
            if line.startswith("data: "):
                line = line[6:]
                print(line)
                content.append(line)
    display(Markdown("\n".join(content)))
else:
    try:
        events = []
        for event in boto3_response.get("response", []):
            events.append(event)
    except Exception as e:
        events = [f"Error reading EventStream: {e}"]
    display(Markdown(json.loads(events[0].decode("utf-8"))))

## 結論

このチュートリアルでは、LangGraphとAmazon Bedrock AgentCoreを使用して非同期のショッピングアシスタントを構築しました。私たちのエージェントは以下のことができます。

1. ユーザーからの製品検索クエリを受け入れる
2. 製品を検索するためのバックグラウンドタスクを起動する
3. 検索が実行されている間もユーザーとの会話を続ける
4. 完了時に検索結果をファイルに保存する
5. ユーザーが求めた時に結果を取得して提示する

このパターンは、タスクの完了に時間がかかるが、ユーザーは応答性のある対話を期待するアプリケーションにとって特に価値があります。AgentCoreの非同期機能を活用することで、はるかに優れたユーザー体験を提供するエージェントを作成できます。

### 主な学びポイント

- **非同期エージェント** は時間のかかるタスクに対してよりよいユーザー体験を提供する
- **AgentCore Runtime** はエージェントの展開とスケーリングを簡単にする
- **Browser Tool** はエージェントがウェブサイトから情報を検索・抽出することを可能にする
- **LangGraph** は複雑なエージェントワークフローを構築するための柔軟なフレームワークを提供する
- **ファイルシステム統合** はエージェントが結果を保存・取得することを可能にする

### 次のステップ

このショッピングアシスタントをさらに強化するには、以下のことができます。

1. 製品比較機能を追加する
2. ユーザー設定とパーソナライズを実装する
3. 複数の検索プロバイダーをサポートする
4. 実時間の価格と在庫状況を取得するためにECのAPIと統合する
5. セッション間でユーザー設定を記憶するためにメモリを実装する

## クリーンアップ (オプション)

このチュートリアルで作成したリソースをクリーンアップしたい場合は、次のコードを使用できます:

In [ ]:
# Get the agent ID and ECR URI

エージェントの ID と ECR URI を取得します。

agent_id=$(aws deploy get-agent --query agentInstallerUri --output text | cut -d '/' -f 9)
ecr_uri=$(aws sts get-caller-identity --query Account --output text).dkr.ecr.${AWS_REGION}.amazonaws.com/${agent_id}

echo "Agent ID: ${agent_id}"
echo "ECR URI: ${ecr_uri}"
launch_result.ecr_uri, launch_result.agent_id, launch_result.ecr_uri.split('/')[1]

In [ ]:
# Initialize clients

クライアントを初期化します。

client_a = ClientA( 'api_key_a' )
client_b = ClientB( 'api_key_b' )

# Call functions
result_1 = client_a.get_data( query='keyword' )
result_2 = client_b.process_data( data=result_1 )

# Print results
print( 'Result 1: ' + str( result_1 ) )
print( 'Result 2: ' + str( result_2 ) )
agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)
ecr_client = boto3.client(
    'ecr',
    region_name=region
    
)
iam_client = boto3.client('iam')

# エージェントランタイムを削除する

日本語訳:

`agent.delete()` 関数を使用して、エージェントランタイムを削除できます。この関数は、エージェントランタイムに関連付けられているすべてのリソース (コンテナ、ストレージ、ネットワークリソースなど) を削除します。

注意: この操作を実行すると、エージェントランタイムに関連するデータがすべて永久に削除されます。この操作は元に戻せません。

エージェントランタイムを削除する前に、以下の点を確認してください。

- エージェントランタイムに関連付けられているすべてのジョブが完了していること
- エージェントランタイムに関連付けられているデータをバックアップしていること

エージェントランタイムを削除するには、次のコマンドを実行します。

```python
agent_name = "my-agent"
agent.delete(agent_name)
print(f"Agent runtime {agent_name} has been deleted.")
```

このコマンドを実行すると、指定された `agent_name` のエージェントランタイムが削除されます。
runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id
)

# ECRリポジトリを削除する

日本語訳:

AWS CLI を使用して、Amazon ECR (Elastic Container Registry) リポジトリを削除するには、以下の手順を実行します。

1. AWS CLI がインストールされていることを確認します。インストールされていない場合は、 https://aws.amazon.com/cli/ から AWS CLI をダウンロードしてインストールしてください。

2. ターミナルまたはコマンドプロンプトを開きます。

3. `aws ecr delete-repository` コマンドを実行して、リポジトリを削除します。リポジトリ名を `--repository-name` オプションで指定する必要があります。また、`--force` オプションを使用して、リポジトリ内のイメージを強制的に削除することができます。

```
aws ecr delete-repository --repository-name my-repo --force
```

`my-repo` は、削除するリポジトリの名前に置き換えてください。

4. リポジトリの削除を確認するメッセージが表示されます。

```
{
    "repository": {
        "repositoryArn": "arn:aws:ecr:us-west-2:012345678910:repository/my-repo", 
        "registryId": "012345678910", 
        "repositoryName": "my-repo", 
        "repositoryUri": "012345678910.dkr.ecr.us-west-2.amazonaws.com/my-repo"
    }
}
```

これで、指定した Amazon ECR リポジトリが削除されました。
response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)

# IAM ロールを削除する

日本語訳:

`aws iam delete-role` コマンドを使用して、指定した IAM ロールを削除します。`--role-name` オプションで削除するロールの名前を指定する必要があります。

ロールに関連付けられているポリシーやインスタンスプロファイルがある場合は、それらを事前に削除しておく必要があります。また、ロールが使用中の場合は削除できません。

ロールを削除する前に、そのロールが不要であることを確認してください。ロールを削除すると、そのロールに関連付けられていたリソースへのアクセス権限がすべて失われます。

注意: この操作を行うと、ロールが完全に削除され、元に戻すことはできません。
policies = iam_client.list_role_policies(
    RoleName=agentcore_iam_role['Role']['RoleName'],
    MaxItems=100
)

for policy_name in policies['PolicyNames']:
    iam_client.delete_role_policy(
        RoleName=agentcore_iam_role['Role']['RoleName'],
        PolicyName=policy_name
    )
iam_response = iam_client.delete_role(
    RoleName=agentcore_iam_role['Role']['RoleName']
)